In [28]:
from selenium import webdriver
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import re

In [29]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [30]:
# technology jobs in the USA
url1 = 'https://jobs.jnj.com/en/jobs/?page='
url2 = '&search=technology&country=United+States&pagesize=20#results'

In [31]:
def jnj_get_jobs(url1, url2):
    """
    retrieve job title and job link
    """
    job_title=[]
    job_link=[]
    page_num = 1

    driver=webdriver.Chrome(options = chrome_options)
    URL = url1 + str(page_num) + url2
    driver.get(URL)
    time.sleep(1)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    while soup.find_all("a", {"class": "stretched-link js-view-job"}) :
        job_title.extend([td.text for td in soup.find_all("a", {"class": "stretched-link js-view-job"})])
        job_link.extend(['https://jobs.jnj.com' + td.parent.find('a')['href'] for td in soup.find_all("a", {"class": "stretched-link js-view-job"})])
        driver.quit()

        page_num += 1
        driver=webdriver.Chrome(options = chrome_options)
        URL = url1 + str(page_num) + url2
        driver.get(URL)
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    driver.quit() 
    return job_title,job_link

In [32]:
job_title, job_link = jnj_get_jobs(url1, url2)

In [33]:
def remove_space(s):
  s_new = s.replace('\n', '')
  while '  ' in s_new:
    s_new = s_new.replace('  ', ' ')
  return s_new

In [34]:
job_title = [remove_space(job) for job in job_title]

In [35]:
# create a dataframe that contains job titles and links for all job categories
df_title_link = pd.DataFrame(zip(job_title, job_link), columns=['JOB_TITLE', 'JOB_LINK'])

In [36]:
# drop the duplicates
df_title_link = df_title_link.drop_duplicates()

1035


## Extract job descriptions and qualifications for each job

In [39]:
# retrieve job qualifications and descriptions
def job_description(title, link):
    qualifications = []
    description = []
    jobtitle = []
    joblink = []
    
    driver=webdriver.Chrome('chromedriver',options=chrome_options)
    for i in range(len(link)):
        URL=link[i]
        driver.get(URL)
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        s = ''
        d = ''
        
        text_pattern = text=re.compile("qualifications", re.IGNORECASE)
       
        # get job qualifications
        try:
          tags = soup.find('h3', text=text_pattern).parent.find_all(['ul', 'ol'])
          if len(tags) > 0:
                for t in tags:
                    s = s + " " + t.text
          else:
              try:
                tags = soup.find('', text=text_pattern).parent.findNextSiblings(['p'])
                for t in tags:
                  s = s + " " + t.text
              except: pass
        except: pass
            
        # get job descriptions
        try:
          tag = soup.find("h2", text='Description').findNext(['ul', 'ol'])
          d = tag.text
        except: pass
        
        qualifications.append(s)
        description.append(d)   
        jobtitle.append(title[i])
        joblink.append(link[i])
        
    driver.quit()            
 
    return jobtitle, link, qualifications, description

In [40]:
# retrieve the qualification for each job.
title, link, qual, descrp = job_description(df_title_link['JOB_TITLE'].values, df_title_link['JOB_LINK'].values)

In [41]:
qual_cleaned = [remove_space(q) for q in qual]
descrp_cleaned = [remove_space(d) for d in descrp]

In [42]:
#create a dataframe from basic qualification and preferred qualification
df = pd.DataFrame(zip(title, qual_cleaned, link, descrp_cleaned))
df.columns = ['TITLE', 'QUALIFICATIONS', 'LINK', 'DESCRIPTION']
df['COMPANY'] = 'JnJ'
df = df.iloc[:, [4, 0, 1, 2, 3]]

In [43]:
# look for empty QUALIFICATION entries
def get_empty(df, colname) :
  empty_idx=[]
  for i in range(len(df[colname])):
    count = len(df[colname][i])
    if count<=10 : empty_idx.append(i)
    else: pass
  return empty_idx

empty_mini_q = get_empty(df, 'QUALIFICATIONS')
print(len(empty_mini_q))

# drop the empty Qualification entries
df.drop(empty_mini_q, inplace=True)

129


In [44]:
#remove the duplicated jobs
df_nodup = df.drop_duplicates()
print("There are {} jobs from JnJ.".format(df_nodup.shape[0]))

df_nodup.to_csv('jnj_technology_jobs_cleaned.csv')

There are 906 jobs from JnJ.
